Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
from stockanalyzer import analyze_stock, stock_data_setup
import importlib

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

# Initialize an empty list to store the stocks
stocks_list = []

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

### Stocks Data

In [ ]:
analysis = analyze_stock(
	ticker= ['MSFT'], 
	dic_files=dic_files,
	dic_assistants=dic_assistants
)

In [5]:
print(analysis)

NameError: name 'analysis' is not defined

In [3]:
ticker = ['MRNA']

client = OpenAI()

file_stocks = stock_data_setup(client=client, ticker=ticker, type='price', dic_files=dic_files)
file_cashflow = stock_data_setup(client=client, ticker=ticker, type='cash', dic_files=dic_files)
file_income_stmt = stock_data_setup(client=client, ticker=ticker, type='income', dic_files=dic_files)

# Have to manually update the tool_resources because the file_id can change
dic_assistants['fin_analyst']['tool_resources'] = {
	'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
}

with open('config/assistants.json', 'w') as json_file:
	json.dump(dic_assistants, json_file)
	print(f'assistants.json file has been updated')

fin_analyst = AgentHandler(
	client = client, 
	new=False,
	assistant_name = 'fin_analyst',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

fin_consultant = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'fin_consultant',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_stocks.csv, file id: file-UpgCVvHFUMJSRs1f96BN1bZP has been deleted
file name: df_stocks.csv is uploaded, new file id: file-5GywACxBfa8Cy7MV2WcYxTmG
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_cashflow.csv, file id: file-YPYRZH07u1bteWlFc4XvmqUm has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-7gS8ZTTWCfnb36afNOv2VYTZ
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_income_stmt.csv, file id: file-cOHsHPwNYXvVrvXsJ4QBV9hH has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-GWhmoDFUCGWjmAzVuzu3cVBM
openai_upload_files/openai_files.json file has been updated
assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U


In [4]:
prompt_start = f"This is the financial consultant.\
	The client wants advice on the following stocks.\
	I need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\
	You will be provided with the data of these stocks in .csv format. Use them as you see fit.\
	Stocks: {ticker}"

thread = ThreadManager(
	client=client,
	prompt=prompt_start
)

thread.run_thread(
	assistant=fin_analyst
	# prompt=prompt_start,
	# attachments=[file_stocks.file_id, file_cashflow.file_id, file_income_stmt.file_id]
)

next_prompt = thread.last_message

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > Here are the information gathered from the financial analyst: 
These are the stocks the client wants advice on:
- MRNA

I will now analyze the provided data. Let's start by examining the contents of the uploaded .csv file to understand what type of information is available.
assistant > code_interpreter

import pandas as pd

# Load the uploaded CSV file
file_path = '/mnt/data/file-5GywACxBfa8Cy7MV2WcYxTmG'
data = pd.read_csv(file_path)

# Display the first few rows and the column names of the CSV to understand its structure
data.head(), data.columns
assistant > Here are the information gathered from the financial analyst: 
These are the stocks the client wants advice on:
- MRNA

The dataset provided includes the following columns for MRNA:
- Date
- Open
- High
- Low
- Close
- Volume
- Dividends
- Stock Splits

Let's analyze the historical price data, focusing on general trends, volatility, and key metrics like average closing price and volume.# Convert the 'Date' column to 

In [9]:
next_prompt = 'This message is for testing. Can you give a short sample response.'

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)

print(thread.last_message)

This message is for testing. Can you give a short sample response.


In [19]:
thread.df_messages
client.beta.threads.messages.list(thread_id=thread.thread_id).data
# messages_data[0]
# type(file_cashflow)

[Message(id='msg_4zwZaBySjeBDhBDOLAECICaH', assistant_id='asst_mPlBQ4ZF8YIF238DWqWRhV4U', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Based on the provided data and analysis, here is a concise assessment and recommendation for MRNA:\n\n**Stock Analysis for MRNA:**\n\n1. **Closing Price Statistics**:\n    - **Mean Closing Price**: $118.85\n    - **Median Closing Price**: $115.99\n    - **Standard Deviation**: $88.50 (indicative of high price volatility)\n    - **Price Range**: $12.26 (min) to $484.47 (max)\n    - **High Volatility**: Annualized volatility at 73.42%, suggesting significant price swings.\n\n2. **Trading Volume**:\n    - **Average Volume**: Approximately 8,078,440 shares, signaling high liquidity.\n\n3. **Date Range**:\n    - **Coverage**: From December 7, 2018, to September 13, 2024.\n\n**Strengths**:\n- **High Liquidity**: The large average trading volume ensures ease of buying and selling the stock.\n- **Growth Potential

In [8]:
thread.get_last_message()

get_last_message initiated


UnboundLocalError: local variable 'previous_message' referenced before assignment

In [17]:
# existing_message_id = [dic['message_id'] for dic in thread.messages]

a = thread.messages[0]

thread.messages = []

thread.messages.append(a)

thread.get_last_message()
# thread.messages[-1]

get_last_message initiated


UnboundLocalError: local variable 'previous_message' referenced before assignment

In [7]:
thread.delete_thread()

thread: thread_RYQYLhm5hagG6NkuutXZYPJG has been deleted.


In [15]:
test = [0, 1, 2, 3, 4]
test[thread.df_messages['_msg_loc'].max()+1:]

TypeError: slice indices must be integers or None or have an __index__ method